# Matrix Factorization Model

* Preprocessing of Dataset
* Create Model
* Model Tuning
* Create Final Model and Prediction

# Import Necessary Libraries

In [1]:
!pip install surprise

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", 500)
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163008 sha256=ed9a3bf07469e2a778bba8fdc3e53abbeb2cfd09c613351a2991b12bee274aad
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


# Import Dataset

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/datasets/movie_dataset.csv")
df.head()

,Unnamed: 0,movieId,title,genres,userId,rating,timestamp
0,2457839,356,Forrest Gump (1994),Comedy|Drama|Romance|War,4.0,4.0,1996-08-24 09:28:42
1,2457840,356,Forrest Gump (1994),Comedy|Drama|Romance|War,7.0,4.0,2002-01-16 19:02:55
2,2457841,356,Forrest Gump (1994),Comedy|Drama|Romance|War,8.0,5.0,1996-06-05 13:44:19
3,2457842,356,Forrest Gump (1994),Comedy|Drama|Romance|War,9.0,4.0,2001-07-01 20:26:38
4,2457843,356,Forrest Gump (1994),Comedy|Drama|Romance|War,10.0,3.0,1999-11-25 02:32:02


# Preprocessing of Dataset

In [3]:
user_movie_df = df.pivot_table(index=["userId"], columns=["title"], values=["rating"])

In [4]:
user_movie_df.head()

rating                                                                                          
title  Blade Runner (1982) Cries and Whispers (Viskningar och rop) (1972) Forrest Gump (1994) The Dark Knight (2011)
userId                                                                                                              
1.0                    4.0                                            NaN                 NaN                    NaN
2.0                    5.0                                            NaN                 NaN                    NaN
3.0                    5.0                                            NaN                 NaN                    NaN
4.0                    NaN                                            NaN                 4.0                    NaN
7.0                    NaN                                            NaN                 4.0                    NaN

In [5]:
reader = Reader(rating_scale=(1,5))

In [6]:
data = Dataset.load_from_df(df[["userId", "movieId", "rating"]], reader)

# Create Model

In [8]:
train_data, test_data = train_test_split(data, test_size=0.3)

In [9]:
svd_model = SVD()

In [10]:
svd_model.fit(train_data)

In [11]:
predictions = svd_model.test(test_data)

In [12]:
accuracy.rmse(predictions)

RMSE: 0.9318


0.9318485763654366

In [13]:
df["movieId"].value_counts()

356       66172
541       30526
4422        644
130219        1
Name: movieId, dtype: int64

In [14]:
df[df["userId"]==10]

,Unnamed: 0,movieId,title,genres,userId,rating,timestamp
4,2457843,356,Forrest Gump (1994),Comedy|Drama|Romance|War,10.0,3.0,1999-11-25 02:32:02


In [15]:
svd_model.predict(uid=10, iid=356, verbose=True)

user: 10         item: 356        r_ui = None   est = 3.92   {'was_impossible': False}


Prediction(uid=10, iid=356, r_ui=None, est=3.923043140225698, details={'was_impossible': False})

# Model Tuning

In [ ]:
param_grid = {"n_epochs": [5, 10, 15, 20, 25, 30],
              "lr_all": [0.002, 0.005, 0.007, 0.008, 0.009]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3, n_jobs=-1, joblib_verbose=True)
gs.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   45.8s
